
# Backfill O/U Actuals — melt stats → join into merged

This notebook:

1. Loads **merged** (props + model) and **weekly player stats** (nflverse).
2. Builds stable keys (`name_std`, `team_std`, `player_key`).
3. **Melts** stats (wide → long) to align with `merged.market_std`.
4. Joins on `name_std + market_std` (disambiguates with team + position).
5. Fills `merged.actual_value` and prints an audit.


In [1]:

# --- config ---
SEASON = 2025
WEEK   = 2

# Paths (edit to your repo layout)
MERGED_CSV = "props_with_model_week2.csv"        # input
STATS_FILE = "weekly_player_stats_2025.parquet"        # input (nflverse weekly)
OUT_MERGED = "merged_with_actuals_week2.csv"      # output (optional)

# If you prefer CSV for stats (fallback)
# STATS_FILE = "data/weekly_player_stats_2025.csv.gz"


In [2]:

import pandas as pd, numpy as np, unicodedata, re, os
from pathlib import Path

def std_name(s: str) -> str:
    """Normalize to ascii, lowercase, strip non-letters, no spaces."""
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode("ascii")
    return re.sub(r"[^a-z]", "", s.lower())

def pick(cols, options):
    """Return first item from `options` that appears in `cols`, else None."""
    return next((c for c in options if c in cols), None)

def ensure_stats_keys(stats: pd.DataFrame) -> pd.DataFrame:
    """Add name_std, team_std, position, player_key (preferring player_id)."""
    df = stats.copy()
    name_col = pick(df.columns, ["player_display_name", "player_name", "player"])
    if not name_col:
        raise KeyError("stats missing player_display_name/player_name")
    df["name_std"] = df[name_col].map(std_name)

    team_col = pick(df.columns, ["recent_team", "team", "posteam"])
    df["team_std"] = df[team_col].astype(str).str.upper() if team_col else pd.NA

    pos_col = pick(df.columns, ["position", "pos", "position_group"])
    df["position"] = df[pos_col].astype(str).str.upper() if pos_col else "UNK"

    # Prefer stable IDs for player_key if present
    id_col = pick(df.columns, ["player_id", "gsis_id", "nfl_id", "pfr_id", "pfr_player_id"])
    if id_col:
        df["player_key"] = df[id_col].astype(str)
    else:
        # Fallback surrogate (may not be unique across seasons): name + team
        df["player_key"] = df["name_std"] + df["team_std"].fillna("")
    return df

def ensure_merged_keys(merged: pd.DataFrame) -> pd.DataFrame:
    """Make sure merged has name_std and game team labels for disambiguation."""
    df = merged.copy()
    if "name_std" not in df.columns:
        name_col = pick(df.columns, ["player", "player_display_name", "player_name", "name"])
        if not name_col:
            raise KeyError("merged lacks name column to build name_std")
        df["name_std"] = df[name_col].map(std_name)
    # game teams
    for c in ["home_team", "away_team"]:
        if c not in df.columns:
            raise KeyError(f"merged missing required column: {c}")
    df["home_team_std"] = df["home_team"].astype(str).str.upper()
    df["away_team_std"] = df["away_team"].astype(str).str.upper()
    return df

# Markets map (normalized names in your pipeline → stats columns)
OU_MAP = {
    "pass_yds":          "passing_yards",
    "rush_yds":          "rushing_yards",
    "recv_yds":          "receiving_yards",
    "receptions":        "receptions",
    "pass_attempts":     "attempts",
    "pass_completions":  "completions",
    "rush_attempts":     "carries",
    "pass_tds":          "passing_tds",
    "pass_ints":         "passing_interceptions",  # nflverse field name
    # Uncomment these if present in your stats file:
    # "reception_longest": "reception_longest",
    # "rush_longest":      "rush_longest",
}

# Position allowlist per market (optional, recommended)
POS_ALLOW = {
    "pass_yds": {"QB"},
    "pass_attempts": {"QB"},
    "pass_completions": {"QB"},
    "pass_tds": {"QB"},
    "pass_ints": {"QB"},
    "rush_yds": {"RB"},
    "rush_attempts": {"RB"},
    # "rush_longest": {"RB"},
    "recv_yds": {"WR","TE"},
    "receptions": {"WR","TE"},
    # "reception_longest": {"WR","TE"},
}


In [3]:

# --- load data ---
merged = pd.read_csv(MERGED_CSV)
print("[merged] shape", merged.shape)

# Load stats (parquet or csv.gz)
if str(STATS_FILE).endswith(".parquet"):
    stats = pd.read_parquet(STATS_FILE)
else:
    stats = pd.read_csv(STATS_FILE)

print("[stats] shape (raw)", stats.shape)

# Optional: filter stats to week/season if columns exist
if {"week", "season"}.issubset(stats.columns):
    stats = stats[(stats["week"] == WEEK) & (stats["season"] == SEASON)].copy()
    print("[stats] after week/season filter", stats.shape)


[merged] shape (2673, 24)
[stats] shape (raw) (2041, 114)
[stats] after week/season filter (970, 114)


In [4]:

# --- keys ---
merged = ensure_merged_keys(merged)
stats  = ensure_stats_keys(stats)

print(merged[["player","name_std","home_team","away_team"]].head())
print(stats[["player_id","player_display_name","name_std","team_std","position"]].head())


         player     name_std       home_team             away_team
0  Bucky Irving  buckyirving  Houston Texans  Tampa Bay Buccaneers
1    Nick Chubb    nickchubb  Houston Texans  Tampa Bay Buccaneers
2  Nico Collins  nicocollins  Houston Texans  Tampa Bay Buccaneers
3    Mike Evans    mikeevans  Houston Texans  Tampa Bay Buccaneers
4  Emeka Egbuka  emekaegbuka  Houston Texans  Tampa Bay Buccaneers
       player_id player_display_name        name_std team_std position
1071  00-0023459       Aaron Rodgers    aaronrodgers      PIT       QB
1072  00-0023853         Matt Prater      mattprater      BUF        K
1073  00-0025565           Nick Folk        nickfolk      NYJ        K
1074  00-0026158          Joe Flacco       joeflacco      CLE       QB
1075  00-0026190     Calais Campbell  calaiscampbell      ARI       DE


In [5]:

# --- melt stats wide->long aligned to market_std ---
have_cols = [col for col in OU_MAP.values() if col in stats.columns]
if not have_cols:
    raise RuntimeError("No O/U stat columns found in stats. Check OU_MAP and stats columns.")

inv_map = {v: k for k, v in OU_MAP.items() if v in have_cols}

s_long = (
    stats[["name_std","team_std","position"] + have_cols]
      .melt(id_vars=["name_std","team_std","position"],
            value_vars=have_cols,
            var_name="stat_col",
            value_name="actual_value")
      .assign(market_std=lambda d: d["stat_col"].map(inv_map))
      .drop(columns=["stat_col"])
      .dropna(subset=["market_std"])
)

print("[s_long] shape", s_long.shape)
display(s_long.head(10))


[s_long] shape (8730, 5)


,name_std,team_std,position,actual_value,market_std
0,aaronrodgers,PIT,QB,203,pass_yds
1,mattprater,BUF,K,0,pass_yds
2,nickfolk,NYJ,K,0,pass_yds
3,joeflacco,CLE,QB,199,pass_yds
4,calaiscampbell,ARI,DE,0,pass_yds
5,matthewstafford,LA,QB,298,pass_yds
6,grahamgano,NYG,K,0,pass_yds
7,thomasmorstead,SF,P,0,pass_yds
8,vonmiller,WAS,OLB,0,pass_yds
9,cameronjordan,NO,DE,0,pass_yds


In [12]:
t1=s_long

In [33]:
t2=merged[['name_std','market_std','model_prob','mu', 'sigma', 'lam','mkt_prob',]]

In [18]:
first_check=t1.merge(t2,how='left', on=['name_std','market_std'])

In [43]:
t1[(t1.market_std=='pass_yds')&(t1.actual_value>0)]

,name_std,team_std,position,actual_value,market_std
0,aaronrodgers,PIT,QB,203,pass_yds
3,joeflacco,CLE,QB,199,pass_yds
5,matthewstafford,LA,QB,298,pass_yds
11,tyrodtaylor,NYJ,QB,56,pass_yds
15,russellwilson,NYG,QB,450,pass_yds
79,dakprescott,DAL,QB,361,pass_yds
81,jaredgoff,DET,QB,334,pass_yds
124,mitchelltrubisky,BUF,QB,32,pass_yds
126,patrickmahomes,KC,QB,187,pass_yds
188,masonrudolph,PIT,QB,12,pass_yds


In [50]:
merged[(merged.market_std=='pass_yds')].name_std.unique().tolist()

['bakermayfield', 'cjstroud', 'justinherbert', 'genosmith']

In [38]:
t2[t2.name_std.str.contains('mahomes')]

,name_std,market_std,model_prob,mu,sigma,lam,mkt_prob


In [23]:
merged[['name_std','market_std','model_prob','mu', 'sigma', 'lam','mkt_prob']]

,name_std,market_std,model_prob,mu,sigma,lam,mkt_prob
0,buckyirving,first_td,NaN,NaN,NaN,NaN,0.153846
1,nickchubb,first_td,NaN,NaN,NaN,NaN,0.142857
2,nicocollins,first_td,NaN,NaN,NaN,NaN,0.125000
3,mikeevans,first_td,NaN,NaN,NaN,NaN,0.105263
4,emekaegbuka,first_td,NaN,NaN,NaN,NaN,0.095238
...,...,...,...,...,...,...,...
2668,amonrastbrown,first_td,NaN,NaN,NaN,NaN,0.090909
2669,derrickhenry,anytime_td,0.221199,0.25,NaN,0.25,0.710145
2670,jahmyrgibbs,anytime_td,0.221199,0.25,NaN,0.25,0.534884
2671,zayflowers,anytime_td,0.221199,0.25,NaN,0.25,0.487805


In [16]:
t2.market_std.unique()

array(['first_td', 'anytime_td', 'last_td', 'pass_attempts',
       'pass_completions', 'interceptions', 'pass_tds', 'pass_yds',
       'reception_longest', 'recv_yds', 'receptions', 'rush_attempts',
       'rush_longest', 'rush_yds'], dtype=object)

In [17]:
t1.market_std.unique()

array(['pass_yds', 'rush_yds', 'recv_yds', 'receptions', 'pass_attempts',
       'pass_completions', 'rush_attempts', 'pass_tds', 'pass_ints'],
      dtype=object)

In [ ]:
# --- assume you already have: m (subset of merged O/U rows with _row), s_long (melted stats) ---

import pandas as pd
import numpy as np

# 1) rename the stats' actuals to avoid collisions
right = s_long[["name_std","market_std","team_std","position","actual_value"]].rename(
    columns={"actual_value": "actual_from_stats"}
)

# 2) left join (name_std, market_std); 'line' stays from the left (m)
cand = m.merge(right, on=["name_std","market_std"], how="left")

# 3) keep only candidates whose team matches the game (if team known)
cand = cand[
    cand["team_std"].isna()
    | cand["team_std"].eq(cand["home_team_std"])
    | cand["team_std"].eq(cand["away_team_std"])
]

# 4) (optional) position filter — keep if your s_long has position
# if you defined POS_ALLOW earlier:
# cand = cand[cand.apply(lambda r: r["position"] in POS_ALLOW.get(r["market_std"], {"QB","RB","WR","TE"}) or r["position"]=="UNK", axis=1)]

# 5) resolve duplicates per merged row by picking the highest stat
if not cand.empty:
    cand["_score"] = cand["actual_from_stats"].fillna(-1e15)
    best = (
        cand.sort_values(["_row","_score"], ascending=[True, False])
            .drop_duplicates(["_row"], keep="first")
    )
else:
    best = cand

# 6) fill back into merged (use a Series, not a raw ndarray)
if "actual_value" not in merged.columns:
    merged["actual_value"] = np.nan

mask_fill = best["actual_from_stats"].notna() & best["_row"].notna()
idx = best.loc[mask_fill, "_row"].astype(int)
vals = best.loc[mask_fill, "actual_from_stats"]
fill_series = pd.Series(vals.to_numpy(), index=idx)

merged.loc[idx, "actual_value"] = merged.loc[idx, "actual_value"].combine_first(fill_series)

print(f"[fill] rows filled: {len(fill_series)}")


In [ ]:

# --- simple join on (name_std, market_std), disambiguate by team + position ---
ou_markets = set(inv_map.values())  # same as keys of OU_MAP that exist

# Keep only O/U rows from merged
m = merged[merged["market_std"].isin(ou_markets)].copy()
m["_row"] = m.index



# Join
cand = m.merge(
    s_long[["name_std","market_std","team_std","position","actual_value"]],
    on=["name_std","market_std"],
    how="left"
)

# Team disambiguation: keep only stats rows where team matches one of the game teams (if team known)
team_ok = (
    cand["team_std"].isna()
    | cand["team_std"].eq(cand["home_team_std"])
    | cand["team_std"].eq(cand["away_team_std"])
)
cand = cand[team_ok]

# Position filter (optional; if position is "UNK" we allow it)
cand = cand[ cand["position"].isin(POS_ALLOW.get(cand["market_std"].iloc[0], {"QB","RB","WR","TE"})) | (cand["position"]=="UNK") ]     if not cand.empty else cand

# Resolve duplicates per merged row by choosing the highest actual_value
if not cand.empty:
    cand["_score"] = cand["actual_value"].fillna(-1e15)
    best = (
        cand.sort_values(["_row","_score"], ascending=[True, False])
            .drop_duplicates(["_row"], keep="first")
    )
else:
    best = cand

# Fill actual_value back into merged
if "actual_value" not in merged.columns:
    merged["actual_value"] = np.nan

mask_fill = best["actual_value"].notna()

# rows we’re filling
idx = best.loc[mask_fill, "_row"]
vals = best.loc[mask_fill, "actual_value"]

# align vals to the merged index we’re writing into
fill_series = pd.Series(vals.to_numpy(), index=idx)

merged.loc[idx, "actual_value"] = (
    merged.loc[idx, "actual_value"].combine_first(fill_series)
)


print("[fill] rows filled:", int(mask_fill.sum()))


In [ ]:

# --- audit ---
def audit_ou(df: pd.DataFrame, markets: set) -> pd.DataFrame:
    out = (df[df["market_std"].isin(markets)]
           .assign(has_line=lambda d: d["line"].notna(),
                   has_actual=lambda d: d["actual_value"].notna())
           .groupby("market_std")
           .agg(rows=("market_std","size"),
                with_line=("has_line","sum"),
                with_actual=("has_actual","sum"))
           .assign(still_missing=lambda d: d["with_line"] - d["with_actual"])
           .sort_index())
    return out

display(audit_ou(merged, ou_markets))

# Save (optional)
Path(Path(OUT_MERGED).parent).mkdir(parents=True, exist_ok=True)
merged.to_csv(OUT_MERGED, index=False)
print("Wrote:", OUT_MERGED)
